In [75]:
import pandas as pd

In [76]:
df = pd.read_csv("all_data.csv")
df['main_label'] = None
df['sub_label'] = None

In [77]:
len(df)

251405

In [78]:
len(
    df[(df["balanceInUSD"] == 0) & (df["depositInUSD"] == 0) & (df["borrowInUSD"] == 0)]
)

1571

In [93]:
max(df['numberOfLiquidation'])

58.0

In [102]:
Q1 = df[df['numberOfInteractedDapps']>0]['numberOfInteractedDapps'].quantile(0.25)
Q3 = df[df['numberOfInteractedDapps']>0]['numberOfInteractedDapps'].quantile(0.95)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [103]:
print("Phần tư thứ nhất (Q1):", Q1)
print("Phần tư thứ ba (Q3):", Q3)
print("IQR:", IQR)
print("Ngưỡng ngoại lai dưới:", lower_bound)
print("Ngưỡng ngoại lai trên:", upper_bound)

# Tìm các giá trị ngoại lai
outliers = df[(df['balanceInUSD'] < lower_bound)]
print("Các giá trị ngoại lai:")
print(outliers)

Phần tư thứ nhất (Q1): 2.0
Phần tư thứ ba (Q3): 18.0
IQR: 16.0
Ngưỡng ngoại lai dưới: -22.0
Ngưỡng ngoại lai trên: 42.0
Các giá trị ngoại lai:
Empty DataFrame
Columns: [_id, address, depositInUSD, borrowInUSD, balanceInUSD, numberCallsTopContracts, totalValueOfLiquidation, numberOfLiquidation, frequencyOfDappTransactions, numberOfInteractedDapps, typesOfInteractedDapps, numberOfReputableDapps, averageTotalAsset, frequencyOfTransaction, frequencyMountOfTransaction, createdAt, label, main_label, sub_label]
Index: []


In [81]:
labels = df[df['balanceInUSD']>-1].get('label')
label_counts = labels.value_counts()
print(label_counts)

label
1    234802
2     12972
4      1876
0       882
3       872
Name: count, dtype: int64


In [82]:
df = df[(df['balanceInUSD'] > 1.0) & (df['createdAt'] != None)]

In [83]:
wallets_to_remove = []
count = 0

In [84]:
for index, row in df.iterrows():
    balanceInUSD = row["balanceInUSD"]
    numberOfLiquidation = row["numberOfLiquidation"]
    borrowInUSD = row["borrowInUSD"]
    # Label 0
    if balanceInUSD <= 1000 and numberOfLiquidation > 0:
        df.loc[index, "main_label"] = 0
    if 1000 < balanceInUSD <= 100000 and borrowInUSD / balanceInUSD >= 0.6:
        df.loc[index, "main_label"] = 0
        df.loc[index, "sub_label"] = 1
    if (
        balanceInUSD > 0
        and borrowInUSD / balanceInUSD >= 0.4
        and numberOfLiquidation > 0
    ):
        df.loc[index, "main_label"] = 0
        df.loc[index, "sub_label"] = 1

    # Label 1
    if balanceInUSD <= 1000 and numberOfLiquidation == 0:
        df.loc[index, "main_label"] = 1
    if (
        0 < balanceInUSD <= 100000
        and 0.4 <= borrowInUSD / balanceInUSD < 0.6
        and numberOfLiquidation == 0
    ):
        df.loc[index, "main_label"] = 1
        df.loc[index, "sub_label"] = 2
    if (
        balanceInUSD > 1000
        and numberOfLiquidation > 0
        and 0 < borrowInUSD / balanceInUSD < 0.4
    ):
        df.loc[index, "main_label"] = 1
        df.loc[index, "sub_label"] = 0

    # Label 2
    if 1000 < balanceInUSD < 10000 and numberOfLiquidation == 0:
        df.loc[index, "main_label"] = 2
        df.loc[index, "sub_label"] = 3
    if (
        balanceInUSD > 1000
        and 0.3 <= borrowInUSD / balanceInUSD < 0.4
        and numberOfLiquidation == 0
    ):
        df.loc[index, "main_label"] = 2

    # Label 3

    if (
        balanceInUSD > 1000000
        and numberOfLiquidation == 0
        and 0.2 <= borrowInUSD / balanceInUSD < 0.3
    ):
        df.loc[index, "main_label"] = 3
    if (
        10000 <= balanceInUSD < 100000
        and 0 < borrowInUSD / balanceInUSD < 0.2
        and numberOfLiquidation == 0
    ):
        df.loc[index, "main_label"] = 3
        df.loc[index, "sub_label"] = 4

    # Label 4
    if (
        balanceInUSD > 1000000
        and numberOfLiquidation == 0
        and borrowInUSD / balanceInUSD < 0.2
    ):
        df.loc[index, "main_label"] = 4
    if balanceInUSD >= 100000 and borrowInUSD == 0 and numberOfLiquidation == 0:
        df.loc[index, "main_label"] = 4
        df.loc[index, "sub_label"] = 3

In [89]:
df.head(6)

,_id,address,depositInUSD,borrowInUSD,balanceInUSD,numberCallsTopContracts,totalValueOfLiquidation,numberOfLiquidation,frequencyOfDappTransactions,numberOfInteractedDapps,typesOfInteractedDapps,numberOfReputableDapps,averageTotalAsset,frequencyOfTransaction,frequencyMountOfTransaction,createdAt,label,main_label,sub_label
2,0x1_0xaa0ff384700c75dbf5e1804da1adac6ab67d0a69,0xaa0ff384700c75dbf5e1804da1adac6ab67d0a69,108.156901,0.000000,3.211923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.019729,0.000000,0.000000,1.687370e+09,1,1,None
3,0x1_0xaa1062ff44cb41b9aac4879dda6e7c21c1e8aa3b,0xaa1062ff44cb41b9aac4879dda6e7c21c1e8aa3b,0.771910,0.000000,17.120238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.187915,0.000000,0.000000,1.629281e+09,1,1,None
4,0x1_0xaa10a6f064cd2daab21633911ef51a72868b0b61,0xaa10a6f064cd2daab21633911ef51a72868b0b61,36.973035,144.087281,17.875705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.156659,0.000000,0.000000,1.630235e+09,1,1,None
13,0x1_0xaa13905714dc09e1c51926869ddf2f3f2f1f3133,0xaa13905714dc09e1c51926869ddf2f3f2f1f3133,0.000104,0.000000,6833.284478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14609.181290,0.000000,66.686542,1.631148e+09,1,2,3
15,0x1_0xaa13d3575491df2721e43bb66cd4a27cd0af725e,0xaa13d3575491df2721e43bb66cd4a27cd0af725e,23.814108,0.000000,3.848197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.122038,0.000000,0.000000,NaN,1,1,None
16,0x1_0xaa13d495f1bf23cf26b7bb37ef35b2a341bce2f4,0xaa13d495f1bf23cf26b7bb37ef35b2a341bce2f4,168.414839,0.000000,2599.080322,0.0,0.0,0.0,5.0,4.0,3.0,2.0,8822.283108,0.133333,55.010699,NaN,2,2,3


In [86]:
# for index, row in df.iterrows():
#     balanceInUSD = row["balanceInUSD"]
#     numberOfLiquidation = row["numberOfLiquidation"]
#     borrowInUSD = row["borrowInUSD"]
#     label = row["label"]
#     if label == 0:
#         if (
#             not (balanceInUSD <= 1000 and numberOfLiquidation > 0)
#             and not (
#                 1000 < balanceInUSD <= 100000 and borrowInUSD / balanceInUSD >= 0.6
#             )
#             and not (
#                 balanceInUSD > 0
#                 and borrowInUSD / balanceInUSD >= 0.4
#                 and numberOfLiquidation > 0
#             )
#         ):
#             wallets_to_remove.append(index)

#     if label == 1:
#         if (
#             not (balanceInUSD <= 1000 and numberOfLiquidation == 0)
#             and not (
#                 balanceInUSD <= 100000
#                 and 0.4 <= borrowInUSD / balanceInUSD < 0.6
#                 and numberOfLiquidation == 0
#             )
#             and not (
#                 balanceInUSD > 1000
#                 and numberOfLiquidation > 0
#                 and 0 < borrowInUSD / balanceInUSD < 0.4
#             )
#         ):
#             wallets_to_remove.append(index)

#     if label == 2:
#         if not (1000 < balanceInUSD < 10000 and numberOfLiquidation == 0) and not (
#             balanceInUSD > 1000
#             and 0.3 <= borrowInUSD / balanceInUSD < 0.4
#             and numberOfLiquidation == 0
#         ):
#             wallets_to_remove.append(index)

#     if label == 3:
#         if not (
#             balanceInUSD > 1000000
#             and numberOfLiquidation == 0
#             and 0.2 <= borrowInUSD / balanceInUSD < 0.3
#         ) and not (
#             10000 <= balanceInUSD < 100000
#             and 0 < borrowInUSD / balanceInUSD < 0.2
#             and numberOfLiquidation == 0
#         ):
#             wallets_to_remove.append(index)
#     if label == 4:
#         if not (
#             balanceInUSD > 1000000
#             and numberOfLiquidation == 0
#             and borrowInUSD / balanceInUSD < 0.2
#         ) and not (
#             balanceInUSD >= 100000 and borrowInUSD == 0 and numberOfLiquidation == 0
#         ):
#             wallets_to_remove.append(index)

In [87]:
df = df.drop(wallets_to_remove)

In [88]:
len(df)

166056